<a href="https://colab.research.google.com/github/hrbjoern/2024-09_CucumberIntelligence/blob/main/notebooks/Finetuning_YOLO_with_Bboxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning a pretrained YOLO detection model with Bboxes only:

## Imports:

In [1]:
import os
from datetime import datetime
import pytz
#import tensorflow as tf

## Some config:

In [2]:
setupnumber = '_alle' # ONLY for saving the model! <-- TODO: cp noch pythonisieren? :-)
# IMPORTANT: Change 'data_loaded' when changing data setup?!!
# ... well, and _remove_ old data?! Just restart runtime. ;-)

modeltype = "yolov8n.pt"
# Other models:
# "yolov8m-seg.pt"
# "yolov8s.pt"
# "yolov8n.pt"


batchsize = 10
if modeltype == "yolov8s.pt":
    batchsize = 20
if modeltype == "yolov8n.pt":
    batchsize = 120

labeltype = "boxes"
#labeltype = 'segs'

if labeltype == 'segs':
    modeltype = modeltype.replace(".pt", "-seg.pt")

imagesize = 640 # 1280
N_epochs = 300

localtime = datetime.now(pytz.timezone('Europe/Berlin'))
starttime = localtime.strftime("%Y-%m-%d_%H%M")
savestring = "Training date = "+starttime
savestring += "\nSetup = "+setupnumber
savestring += "\nModel = "+modeltype
savestring += "\nLabeltype = "+labeltype
savestring += "\nBatchsize = "+ str(batchsize)
savestring += "\nImagesize = "+ str(imagesize)
savestring += "\nEpochs = " + str(N_epochs)
print(savestring)


Training date = 2024-09-28_2145
Setup = _alle
Model = yolov8n.pt
Labeltype = boxes
Batchsize = 120
Imagesize = 640
Epochs = 300


## Fetch the data:

In [3]:
# Mount Google Drive and copy data:
if 'data_loaded' not in globals():
    from google.colab import drive
    drive.mount('/content/drive')
    !mkdir -v /content/Data

    !cp -r /content/drive/MyDrive/setup_015/* /content/Data/ #| pv
    print("setup015 copied")
    !cp -r /content/drive/MyDrive/setup_002/* /content/Data/
    print("setup002 copied")
    !cp -r /content/drive/MyDrive/setup_003/* /content/Data/
    print("setup003 copied")
    !cp -r /content/drive/MyDrive/setup_006/* /content/Data/
    print("setup006 copied")
    !cp -v /content/drive/MyDrive/setup-002-standalone.yaml /content/Data/

    print("Data download finished at", datetime.now(pytz.timezone('Europe/Berlin')).strftime("%H:%M"))


Mounted at /content/drive
mkdir: created directory '/content/Data'
setup015 copied
setup002 copied
setup003 copied
setup006 copied
'/content/drive/MyDrive/setup-002-standalone.yaml' -> '/content/Data/setup-002-standalone.yaml'
Data download finished at 21:54


## Fetch helper scripts/functions and restructure the data:

In [4]:
if 'data_loaded' not in globals():
    !cp -v /content/drive/MyDrive/prepare_yolo_data_folders.py /content/
    # Download helper script from Github: -> doesn't work.
    #!wget https://raw.githubusercontent.com/<username>/<reponame>/<branch>/prepare_yolo_data_folders.py
    from prepare_yolo_data_folders import prepare_yolo_data_folders

    # Colab magic to make "Data" the current directory:
    %cd /content/Data

    #Do the train-validation split and move the image and label files to YOLO folder structure:
    data_loaded = prepare_yolo_data_folders(labeltype) # TODO: Quiet! :-)
    print('data_loaded is', data_loaded)

'/content/drive/MyDrive/prepare_yolo_data_folders.py' -> '/content/prepare_yolo_data_folders.py'
/content/Data
Preparing files and folders for training with boxes labels.
Current folder is: /content/Data
Found 354 labels.
Found 354 labelled images.
Created train/images and train/labels.
Created val/images and val/labels.
Created test/images and test/labels.
Moved 247 images and labels to folders.
Moved 71 images and labels to folders.
Moved 36 images and labels to folders.
Data handling finished.
data_loaded is True


In [5]:
# Count the files:
num_train = len(os.listdir('/content/Data/train/labels'))
num_val = len(os.listdir('/content/Data/val/labels'))
num_test = len(os.listdir('/content/Data/test/labels'))
print(f"Number of training images: {num_train}")
print(f"Number of validation images: {num_val}")
print(f"Number of test images: {num_test}")

savestring += "\nNumber of training images: " + str(num_train)
savestring += "\nNumber of validation images: " + str(num_val)
savestring += "\nNumber of test images: " + str(num_test)

Number of training images: 247
Number of validation images: 71
Number of test images: 36


### Write settings file:

In [6]:
# Output settings file:
settingsfile = '/content/drive/MyDrive/CucumberModels/'+starttime + "_settings.txt"
with open(settingsfile, "w") as file:
    file.write(savestring)
print("Model settings written to", settingsfile)

Model settings written to /content/drive/MyDrive/CucumberModels/2024-09-28_2145_settings.txt


## Install ultralytics and import YOLO:

In [7]:
%pip -q install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.1/875.1 kB 21.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Okay, so the images and labels are there? Next: Import YOLO model for training!
Note: Check Bboxes vs. segmentations!

## Load tensorboard:

In [8]:
%load_ext tensorboard

## Import YOLO model for training

In [9]:
# Load a pretrained model:
model = YOLO(modeltype)

100%|██████████| 6.25M/6.25M [00:00<00:00, 130MB/s]


### YAML file for folder and class definition: Everything ok?

In [10]:
#!cat /content/misc/setup-002-standalone.yaml
yamlfile = "/content/Data/setup-002-standalone.yaml"

## Do the training!

In [11]:
# If GPU-RAM is full before training:

# import torch
# torch.cuda.empty_cache()


In [12]:
# Train the model
trainresults = model.train(data=yamlfile, epochs=N_epochs, imgsz=imagesize, batch=batchsize)

Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Data/setup-002-standalone.yaml, epochs=300, time=None, patience=100, batch=120, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show

100%|██████████| 755k/755k [00:00<00:00, 23.1MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/Data/train/labels... 247 images, 0 backgrounds, 0 corrupt: 100%|██████████| 247/247 [00:21<00:00, 11.61it/s]

train: New cache created: /content/Data/train/labels.cache


...

    300/300      15.5G     0.4629     0.2819     0.8391         59        640: 100%|██████████| 3/3 [00:01<00:00,  1.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.38it/s]

                   all         71        562       0.98      0.986      0.992      0.846



300 epochs completed in 0.696 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.00s/it]


                   all         71        562       0.98      0.989      0.992      0.847
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/train


### Save the model directly to Drive:

In [13]:
modeldir = '/content/drive/MyDrive/CucumberModels/'+starttime+'_'+labeltype
if not os.path.exists(modeldir):
    os.makedirs(modeldir)

model.save(modeldir+"/"+starttime+".pt")

### wandb?

... and managing the parameter "matrix"

### Tensorboard:

In [14]:
# Show tensorboard:
#%tensorboard --logdir /content/runs/detect #path/to/runs

## Predict the cucumbers on test images:

In [16]:
# Examples:
# #predictresults = model(["image1.jpg", "image2.jpg"])  # return a list of Results objects
# # Run inference on 'bus.jpg' with arguments
# model.predict("bus.jpg", save=True, imgsz=320, conf=0.5)


# /content/Data/test/images

# First, I guess we should _dynamically_?! define a list of images to predict on:
#imagelist = []
imagelist = ['test/images/'+file for file in os.listdir("/content/Data/test/images")]
print(f"Using {len(imagelist)} images for prediction: ")
print(imagelist)

# Then, we can predict on the images:
results = model.predict(imagelist,
                        imgsz=imagesize,
                        conf=0.6,
                        #show=True,
                        save=True)

print("Where are the resulting pictures stored? :-) \n" +
      "And how can I rescale them for/after prediction?")

for index, result in enumerate(results):
  result.save(modeldir+"/"+starttime+"_prediction_"+str(index)+".jpg")

# # Process result:
# # for result in results:
# boxes = result.boxes  # Boxes object for bounding box outputs
# # masks = result.masks  # Masks object for segmentation masks outputs
# # keypoints = result.keypoints  # Keypoints object for pose outputs
# probs = result.probs  # Probs object for classification outputs
# obb = result.obb  # Oriented boxes object for OBB outputs
# result.show()  # display to screen

Using 36 images for prediction: 
['test/images/dsc01856.png', 'test/images/dsc01950.png', 'test/images/20220614_134228.png', 'test/images/dsc01961.png', 'test/images/012.png', 'test/images/dsc01840.png', 'test/images/059.png', 'test/images/dsc01956.png', 'test/images/024.png', 'test/images/20220614_131152.png', 'test/images/20220614_131101.png', 'test/images/20220614_131239.png', 'test/images/025.png', 'test/images/dsc01863.png', 'test/images/dsc01904.png', 'test/images/050.png', 'test/images/058.png', 'test/images/dsc02031.png', 'test/images/20220614_134223.png', 'test/images/20220614_131145_a.png', 'test/images/dsc01833.png', 'test/images/dsc01830.png', 'test/images/dsc01970.png', 'test/images/021.png', 'test/images/022.png', 'test/images/009.png', 'test/images/dsc01969.png', 'test/images/002.png', 'test/images/dsc01850.png', 'test/images/20220614_131128.png', 'test/images/20220614_131157.png', 'test/images/20220614_131233.png', 'test/images/20220614_131125.png', 'test/images/dsc0183

### Show resulting pictures:

Perhaps replace IPython display by PIL stuff?

In [ ]:
#from IPython.display import Image
#Image(filename='/content/runs/detect/train53/20220614_134218_Gurkenfoto_2.jpg', width=400)


In [ ]:
#Image(filename='/content/runs/detect/train2/20220614_134218_Gurkenfoto_rescaled_640.jpg', width=400)


In [ ]:
#Image(filename='/content/runs/detect/train53/dsc01843.png', width=400)